In [1]:
import sys
sys.path.append('..')
import os
os.chdir("..")
import logging
logging.basicConfig(level=logging.WARN)
import ray
ray.init(log_to_driver=False)#
from janusq.analysis.fidelity_prediction import FidelityModel
from janusq.baselines.fidelity_prediction.rb_prediction import RBModel

from janusq.analysis.vectorization import RandomwalkModel

from janusq.data_objects.backend import GridBackend

from janusq.tools.ray_func import map

from janusq.data_objects.circuit import SeperatableCircuit

import numpy as np

2024-04-16 05:30:18,851	WARNING services.py:1996 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2024-04-16 05:30:18,998	INFO worker.py:1724 -- Started a local Ray instance.


In [10]:
import io
import pickle


class RenameUnpickler(pickle.Unpickler):
    def find_class(self, module, name):

        renamed_module = module
        if module.startswith("data_objects"):# or module.startwith("analysis") or module.startwith("data_objects")
            renamed_module = "janusq." + module

        return super(RenameUnpickler, self).find_class(renamed_module, name)


def renamed_load(file_obj):
    return RenameUnpickler(file_obj).load()



import pickle
with open("examples/dataset/fidelity_dataset_18q.pkl", "rb") as f:
    circuits: list[SeperatableCircuit] = renamed_load(f)   # TODO: 整理成circuits一个数组，fidelity一个数组的形式
print(len(circuits))

2500


In [11]:
sub_circuits, sub_fidelities = [], []
for circuit in circuits:
    for sub_cir in circuit.seperatable_circuits:
        sub_circuits.append(sub_cir)
        sub_fidelities.append(sub_cir.ground_truth_fidelity)

print(len(sub_circuits), len(sub_fidelities))

10000 10000


In [12]:
for cir in sub_circuits:
    qubit_map = {}
    for i, qubit in enumerate(cir.operated_qubits):
        qubit_map[qubit] = i
    cir.n_qubits = len(cir.operated_qubits)
    cir.operated_qubits = list(range(cir.n_qubits))
    
    for i, g in enumerate(cir.gates):
        g['id'] = i
        for i in range(len(g['qubits'])):
            g['qubits'][i] = qubit_map[g['qubits'][i]]


In [17]:
target_circutis, target_fidelities = [], []
for i, cir in enumerate(sub_circuits):
    if cir.n_qubits == 5:
        target_circutis.append(cir)
        target_fidelities.append(sub_fidelities[i])
len(target_circutis)

7500

In [18]:
import pickle
pickle.dump((target_circutis, target_fidelities), open("fidelity_dataset_5q.pkl","wb"))